In [3]:
import pandas as pd 
from sklearn.feature_selection import VarianceThreshold
import sknetwork

## Data loading (Simple dataset)
We will load all needed dataset in our program.

In [9]:
users_features = pd.read_csv("data/Social_spammers_dataset/users_features/features.csv")
labels = pd.read_csv("data/Social_spammers_dataset/users/coded_ids_labels_train.csv")
code_ids_label = pd.read_csv("data/Social_spammers_dataset/users/coded_ids.csv")
users_features = pd.merge(users_features, code_ids_label, on='user_id')

In [12]:
users_features_with_labels = pd.merge(users_features, labels, on='coded_id')

In [13]:
users_features_with_labels

,active_tweeting_frequency_per_day,adjusted_nb_of_uses_of_hashtag,adjusted_nb_of_uses_of_mention,adjusted_nb_of_uses_of_sources,adjusted_nb_of_uses_of_url,age,avg_intertweet_times,avg_intertweet_times_seconds,content_duration_days,date_newest_tweet,...,time_zone,tweeting_frequency_per_day,tweets_with_at_top_20,tweets_with_hashtags_top_20,tweets_with_urls_top_20,urls_used_on_average,user_id,utc_offset,coded_id,label
0,0.055,2.600,5.308,21.087,2.333,2186,19 days 05:12:37.409091000,1660357.0,422,26/12/2017 14:45:25,...,Hawaii,5.396,10,5,5,2.000,494199648,-36000.0,203,0
1,40.000,9.938,8.329,400.000,1.000,2474,0 days 00:39:20.897243000,2361.0,10,10/02/2018 17:00:37,...,Riyadh,35.167,19,2,1,1.000,293270197,10800.0,138,0
2,0.334,2.600,2.412,55.284,36.885,3220,16 days 16:04:30.509317000,1440271.0,485,04/07/2011 03:37:09,...,Pacific Time (US & Canada),0.051,0,0,20,2.400,33046208,-28800.0,148,1
3,4.494,0.000,0.000,203.380,1.000,816,0 days 05:24:08.857143000,19449.0,89,09/02/2018 12:43:09,...,Pacific Time (US & Canada),23.863,0,0,20,1.000,4217668515,-28800.0,182,1
4,80.000,395.000,0.000,400.000,174.673,1174,0 days 00:19:59.997494000,1200.0,5,10/02/2018 17:15:00,...,Central Time (US & Canada),46.056,0,19,20,1.908,2909542704,-21600.0,135,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,0.995,1.000,2.256,163.250,1.013,2738,1 days 00:19:44.788945000,87585.0,201,25/10/2017 01:47:41,...,Seoul,1.440,6,0,15,1.006,138282579,32400.0,747,0
682,100.000,42.436,35.122,98.490,1.000,3405,0 days 00:14:56.216080000,896.0,2,25/10/2017 02:22:04,...,Eastern Time (US & Canada),23.174,20,19,1,1.000,15290441,-14400.0,541,0
683,2.020,0.000,0.000,136.000,1.000,2249,0 days 11:56:53.567839000,43014.0,99,25/10/2017 02:45:34,...,Tokyo,2.512,0,0,11,1.000,365303324,32400.0,474,0
684,1.418,1.912,5.804,147.610,1.077,3137,0 days 17:06:45.271357000,61605.0,141,24/10/2017 05:27:02,...,London,7.198,19,8,6,1.040,26738483,3600.0,435,0


## Data Loading (Graph dataset)

In [1]:
graph = pd.read_csv("/data/Social_spammers_dataset/graphs/c_combined_edges_w_1_s_0.9.csv")

## Data preprocessing

## Exploration of data

In [5]:
users_features

,active_tweeting_frequency_per_day,adjusted_nb_of_uses_of_hashtag,adjusted_nb_of_uses_of_mention,adjusted_nb_of_uses_of_sources,adjusted_nb_of_uses_of_url,age,avg_intertweet_times,avg_intertweet_times_seconds,content_duration_days,date_newest_tweet,...,time_since_newest_tweet_days,time_since_newest_tweet_months,time_zone,tweeting_frequency_per_day,tweets_with_at_top_20,tweets_with_hashtags_top_20,tweets_with_urls_top_20,urls_used_on_average,user_id,utc_offset
0,0.055,2.600,5.308,21.087,2.333,2186,19 days 05:12:37.409091000,1660357.0,422,26/12/2017 14:45:25,...,46,1.533333,Hawaii,5.396,10,5,5,2.000,494199648,-36000.0
1,154.333,3.447,13.164,463.000,1.045,1286,0 days 00:10:33.400433000,633.0,3,27/12/2014 22:46:34,...,1141,38.033333,NaN,11.567,13,6,1,1.023,2708449765,NaN
2,40.000,9.938,8.329,400.000,1.000,2474,0 days 00:39:20.897243000,2361.0,10,10/02/2018 17:00:37,...,0,0.000000,Riyadh,35.167,19,2,1,1.000,293270197,10800.0
3,0.334,2.600,2.412,55.284,36.885,3220,16 days 16:04:30.509317000,1440271.0,485,04/07/2011 03:37:09,...,2414,80.466667,Pacific Time (US & Canada),0.051,0,0,20,2.400,33046208,-28800.0
4,4.494,0.000,0.000,203.380,1.000,816,0 days 05:24:08.857143000,19449.0,89,09/02/2018 12:43:09,...,1,0.033333,Pacific Time (US & Canada),23.863,0,0,20,1.000,4217668515,-28800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,200.000,1.540,2.252,200.000,1.000,3221,0 days 00:06:09.909548000,370.0,1,25/10/2017 10:09:26,...,2,0.066667,London,93.877,14,4,12,1.000,18525497,3600.0
763,100.000,42.436,35.122,98.490,1.000,3405,0 days 00:14:56.216080000,896.0,2,25/10/2017 02:22:04,...,2,0.066667,Eastern Time (US & Canada),23.174,20,19,1,1.000,15290441,-14400.0
764,2.020,0.000,0.000,136.000,1.000,2249,0 days 11:56:53.567839000,43014.0,99,25/10/2017 02:45:34,...,2,0.066667,Tokyo,2.512,0,0,11,1.000,365303324,32400.0
765,1.418,1.912,5.804,147.610,1.077,3137,0 days 17:06:45.271357000,61605.0,141,24/10/2017 05:27:02,...,3,0.100000,London,7.198,19,8,6,1.040,26738483,3600.0


In [6]:
labels

,coded_id,label
0,1,1
1,2,0
2,3,0
3,4,1
4,6,0
...,...,...
681,763,0
682,764,0
683,765,0
684,766,0


In [ ]:
VarianceThreshold(threshold=(.8 * (1 - .8)))

## Feature Selection

## Data Analysis part

##  Models

## Evaluation of models